<a href="https://colab.research.google.com/github/Yahred/evolutionary-computation/blob/main/CGXV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de rutas a través de algoritmos genéticos

Se cargan módulos necesarios

In [129]:
pip install deap

ERROR: Could not find a version that satisfies the requirement folium_static (from versions: none)
ERROR: No matching distribution found for folium_static


In [130]:
import json
import math
import random
import folium

import matplotlib.pyplot as plt
import numpy as np

from deap import base, creator, tools, algorithms
from urllib.request import urlopen
from IPython.display import display

Se cargan datos sobre las capitales

In [112]:
path = 'https://raw.githubusercontent.com/Yahred/evolutionary-computation/main/data/capitales.json'
archivo = urlopen(path).read()
data = json.loads(archivo)

CAPITALES = data['capitales']
COORDENADAS = data['coordenadas']
DISTANCIAS = data['distancias']
ESPACIOS = 8
CENTRO_EU = [54.6872, 25.2797]
ZOOM_MAPA = 4

Se calcula la longitud del cromosoma en base a las capitales y el número de espacios disponibles

In [55]:
LONGITUD_CROMOSOMA = 0
for i in range(ESPACIOS):
  LONGITUD_CROMOSOMA += math.ceil(math.log(len(CAPITALES) - i, 2))

LONGITUD_CROMOSOMA

28

In [38]:
CAPITALES

['Madrid',
 'París',
 'Berlín',
 'Roma',
 'Atenas',
 'Lisboa',
 'Varsovia',
 'Praga',
 'Ámsterdam',
 'Viena',
 'Estocolmo',
 'Budapest']

In [39]:
COORDENADAS

{'Madrid': {'latitud': 40.416775, 'longitud': -3.70379},
 'París': {'latitud': 48.856613, 'longitud': 2.352222},
 'Berlín': {'latitud': 52.520008, 'longitud': 13.404954},
 'Roma': {'latitud': 41.902782, 'longitud': 12.496366},
 'Atenas': {'latitud': 37.98381, 'longitud': 23.727539},
 'Lisboa': {'latitud': 38.722252, 'longitud': -9.139337},
 'Varsovia': {'latitud': 52.229676, 'longitud': 21.012229},
 'Praga': {'latitud': 50.075538, 'longitud': 14.4378},
 'Ámsterdam': {'latitud': 52.366696, 'longitud': 4.89454},
 'Viena': {'latitud': 48.208174, 'longitud': 16.373819},
 'Estocolmo': {'latitud': 59.329323, 'longitud': 18.068581},
 'Budapest': {'latitud': 47.497913, 'longitud': 19.040236}}

In [40]:
DISTANCIAS

{'Madrid': {'Madrid': 0,
  'París': 1056,
  'Berlín': 1886,
  'Roma': 2226,
  'Atenas': 2760,
  'Lisboa': 634,
  'Varsovia': 2502,
  'Praga': 1752,
  'Ámsterdam': 1375,
  'Viena': 2275,
  'Estocolmo': 2898,
  'Budapest': 1775},
 'París': {'Madrid': 1056,
  'París': 0,
  'Berlín': 878,
  'Roma': 1105,
  'Atenas': 1842,
  'Lisboa': 1490,
  'Varsovia': 1052,
  'Praga': 1029,
  'Ámsterdam': 431,
  'Viena': 1133,
  'Estocolmo': 1903,
  'Budapest': 1345},
 'Berlín': {'Madrid': 1886,
  'París': 878,
  'Berlín': 0,
  'Roma': 1180,
  'Atenas': 2040,
  'Lisboa': 2154,
  'Varsovia': 265,
  'Praga': 280,
  'Ámsterdam': 657,
  'Viena': 683,
  'Estocolmo': 1618,
  'Budapest': 684},
 'Roma': {'Madrid': 2226,
  'París': 1105,
  'Berlín': 1180,
  'Roma': 0,
  'Atenas': 1584,
  'Lisboa': 1956,
  'Varsovia': 1661,
  'Praga': 1454,
  'Ámsterdam': 1632,
  'Viena': 857,
  'Estocolmo': 2376,
  'Budapest': 1330},
 'Atenas': {'Madrid': 2760,
  'París': 1842,
  'Berlín': 2040,
  'Roma': 1584,
  'Atenas': 0,
  '

Graficamos las capitales que tenemos de opción

In [132]:
mapa = folium.Map(location=CENTRO_EU, zoom_start=ZOOM_MAPA)

for (capital, coord) in COORDENADAS.items():
  lat = coord['latitud']
  lng = coord['longitud']
  folium.Marker([lat, lng], tooltip=capital).add_to(mapa)

mapa

# Definición de la función de la evaluación

In [41]:
class Capital:
  def __init__(self, nombre: str, lat, lng) -> None:
    self.nombre = nombre
    self.lat = lat
    self.lng = lng

  def __str__(self):
    return f'{self.nombre} Latitud: {self.lat} Longitud: {self.lng}'

Definimos una función para gráficar el mapa de las capitales

In [133]:
def graficar_capitales(capitales: list[Capital]):
  mapa = folium.Map(location=CENTRO_EU, zoom_start=ZOOM_MAPA)

  coordenadas = []
  for capital in capitales:
    coordenadas.append((capital.lat, capital.lng))

  linea = folium.PolyLine(coordenadas, color='red')
  linea.add_to(mapa)

  return mapa

In [42]:
def capital_factory(nombre: str):
  coordenadas = COORDENADAS[nombre]
  return Capital(nombre, coordenadas['latitud'], coordenadas['longitud'])

In [43]:
def calcular_distancia(a: Capital, b: Capital):
  distancias = DISTANCIAS[a.nombre]
  return distancias[b.nombre]

In [44]:
def calcular_recorrido(capitales: list[Capital]) -> float:
  distancia = 0

  for i in range(1, len(capitales)):
    a = capitales[i - 1]
    b = capitales[i]
    distancia += calcular_distancia(a, b)

  return distancia

In [86]:
def decode_ind(bin: str) -> list[Capital]:
  opciones = CAPITALES.copy()
  capitales = []
  bin_restante = bin
  for i in range(ESPACIOS):
    n_bits = math.ceil(math.log(len(opciones), 2))
    segmento_bin = bin_restante[:n_bits]
    bin_restante = bin_restante[n_bits:]

    seleccion = int(segmento_bin, 2)

    if seleccion >= len(opciones):
      seleccion = len(opciones) - 1

    capital_seleccionada = capital_factory(opciones.pop(seleccion))
    capitales.append(capital_seleccionada)

  return capitales

In [87]:
def fitness(ind: list[str]):
  capitales = decode_ind(''.join(ind))
  distancia = calcular_recorrido(capitales)
  return distancia,

# Definición de los individuos



In [88]:
def crear_ind():
    return ''.join(random.choice('01') for _ in range(LONGITUD_CROMOSOMA))


creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("individual", tools.initIterate, creator.Individual, crear_ind)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=5)

stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("avg", np.mean)

In [89]:
ind = toolbox.individual()
ind

['1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0']

In [96]:
random.seed(64)
n_gen = 10000
initial_pop = 250

pop = toolbox.population(n=initial_pop)
hof = tools.HallOfFame(1)
pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.8, mutpb=0.2, ngen=n_gen, halloffame=hof, verbose=True, stats=stats)

ganador = tools.selBest(pop, k=1)[0]

Se truncaron las últimas líneas 5000 del resultado de transmisión.
5001	134   	4190.15
5002	124   	4140.3 
5003	139   	4121.57
5004	143   	4285.85
5005	125   	4106.82
5006	124   	4105.34
5007	128   	4198.53
5008	135   	4225.29
5009	154   	4194.25
5010	130   	4108.3 
5011	119   	4301.54
5012	134   	4201.27
5013	128   	4125.37
5014	121   	4292.69
5015	126   	4210.85
5016	138   	4204.08
5017	146   	4151.67
5018	135   	4269.89
5019	157   	4080.06
5020	154   	4306.92
5021	147   	4155.74
5022	137   	4219.94
5023	134   	4250.67
5024	135   	4194.24
5025	117   	4056.02
5026	153   	4216.24
5027	150   	4122.6 
5028	133   	4107.29
5029	139   	4193.5 
5030	133   	4142.51
5031	146   	4312.68
5032	120   	4210.08
5033	138   	4087.15
5034	149   	4193.48
5035	136   	4235.37
5036	141   	4232.58
5037	147   	4057.99
5038	161   	4201.6 
5039	144   	4105.42
5040	142   	4140.37
5041	142   	4161.97
5042	133   	4134.39
5043	124   	4273.93
5044	136   	4150.02
5045	144   	4181.2 
5046	139   	4249.58
5047	156   	4

In [99]:
ganador

['0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0']

In [100]:
distancia, = fitness(ganador)
distancia

3162

In [135]:
capitales = decode_ind(''.join(ind))

In [ ]:
graficar_capitales(capitales)